<br>

# Introdução

In [17]:
import os
import folium
import simplekml
import pandas as pd
import seaborn as sns
import geopandas as gpd
from folium import plugins
#from folium.plugins import FastMarkerCluster
from shapely.geometry import Point

<br>

## Dataframe

In [18]:
# Tabela Pontos de Monitoramento
df = pd.read_csv(
    os.path.join('..', 'data', 'tab_pontos_monitoramento_amostras.csv'),
)

# Seleciona os que tem coordenadas
df = df[(df['tem_coordenadas'] == 0)]

# E que são diferentes de zero!
#df = df[(df['latitude_dd'] != 0)]
#df = df[(df['longitude_dd'] != 0)]

# Results
print('{} pontos\n'.format(len(df)))
print(df.dtypes)
df.head(2)

644 pontos

cod_ponto                 object
cod_interaguas             int64
tipo_rede                 object
status_ponto              object
sistema_hidrico_sigla     object
sistema_hidrico           object
sistema_hidrico_tipo      object
classe                    object
tem_coordenadas            int64
latitude_dd              float64
longitude_dd             float64
latitude_dms              object
longitude_dms             object
altitude                   int64
localizacao               object
captacao                  object
municipio                 object
estado                    object
ugrhi                     object
n_amostras                 int64
dtype: object


,cod_ponto,cod_interaguas,tipo_rede,status_ponto,sistema_hidrico_sigla,sistema_hidrico,sistema_hidrico_tipo,classe,tem_coordenadas,latitude_dd,longitude_dd,latitude_dms,longitude_dms,altitude,localizacao,captacao,municipio,estado,ugrhi,n_amostras
0,ABAN02790,2592,Rede Básica,Inativo,ABAN,Córrego Água Branca,Rio (Lótico),Classe 2,0,-22.231389,-47.853056,22 13 53,47 51 11,725,A montante do Córrego do Limoeiro,N,ITIRAPINA,SP,13 - TIETE/JACARÉ,1
1,ABAN02800,2584,Rede Básica,Ativo,ABAN,Córrego Água Branca,Rio (Lótico),Classe 2,0,-22.231111,-47.853056,22 13 52,47 51 11,725,"Ponte na Rod. Mun. Ayrton Sena, a jusante da E...",N,ITIRAPINA,SP,13 - TIETE/JACARÉ,1013


<br>

## Geojson

In [20]:
# Create Geometry
geometry = [Point(xy) for xy in zip(df['longitude_dd'], df['latitude_dd'])]

# Create Geodataframe
gdf = gpd.GeoDataFrame(
    df,
    crs='EPSG:4326',
    geometry=geometry
)

# Save
output_path = os.path.join('..', 'data', 'geo')
os.makedirs(output_path, exist_ok=True)
gdf.to_file(os.path.join(output_path, 'ptos_monitoramento.geojson'), driver='GeoJSON')
gdf.head(2)

,cod_ponto,cod_interaguas,tipo_rede,status_ponto,sistema_hidrico_sigla,sistema_hidrico,sistema_hidrico_tipo,classe,tem_coordenadas,latitude_dd,...,latitude_dms,longitude_dms,altitude,localizacao,captacao,municipio,estado,ugrhi,n_amostras,geometry
0,ABAN02790,2592,Rede Básica,Inativo,ABAN,Córrego Água Branca,Rio (Lótico),Classe 2,0,-22.231389,...,22 13 53,47 51 11,725,A montante do Córrego do Limoeiro,N,ITIRAPINA,SP,13 - TIETE/JACARÉ,1,POINT (-47.85306 -22.23139)
1,ABAN02800,2584,Rede Básica,Ativo,ABAN,Córrego Água Branca,Rio (Lótico),Classe 2,0,-22.231111,...,22 13 52,47 51 11,725,"Ponte na Rod. Mun. Ayrton Sena, a jusante da E...",N,ITIRAPINA,SP,13 - TIETE/JACARÉ,1013,POINT (-47.85306 -22.23111)


<br>

## KML

In [21]:
kml = simplekml.Kml(open=1)

# Procedimentos
fol_proc = kml.newdocument()
fol_proc.name = 'Pontos de Monitoramento'
fol_proc.description = 'Localização dos Pontos de Monitoramento de Rede Básica da CETESB, disponíveis no InfoÁguas'

#
icone = 'http://maps.google.com/mapfiles/kml/pal3/icon61.png'

# 
for i, row in gdf.iterrows():
    # 
    df_temp = pd.DataFrame(row)
    df_temp = df_temp.to_html(header=False)

    if row['tipo_rede'] == 'Rede Básica':
        pnt = fol_proc.newpoint()
        pnt.name = row['cod_ponto']
        pnt.coords = [(row['longitude_dd'], row['latitude_dd'])]

        pnt.style.labelstyle.scale = 0
        pnt.style.labelstyle.color = simplekml.Color.red

        pnt.style.iconstyle.icon.href = icone
        pnt.style.iconstyle.color = simplekml.Color.red

        pnt.style.balloonstyle.text = '{}'.format(df_temp)
        #pnt.style.balloonstyle.bgcolor = simplekml.Color.purple
        #pnt.style.balloonstyle.textcolor = simplekml.Color.black
        pnt.style.balloonstyle.displaymode = 'default'

    else:
        print('Erro! Tipo de ponto "{}" não definido'.format(row['tipo_rede']))

kml.save(os.path.join(output_path, 'ptos_monitoramento.kml'))
kml.save(os.path.join(os.path.expanduser('~/Documents/Sourcecode'), 'gitpages', 'open-geodata.github.io', 'assets', 'sp_cetesp_infoaguas', 'ptos_monitoramento.kml'))

<br>

# Layers

<br>

## Pontos de Monitoramento

In [5]:
def add_lyr_ptos_monitoramento():
    # Input
    gdf = gpd.read_file(os.path.join('..', 'data', 'geo', 'ptos_monitoramento.geojson'))
    gdf = gdf.to_crs(epsg=4326)
    
    # Popup
    #gdf['popup'] = gdf.apply(popup_html_est_aut_empresas, axis=1)

    # Layer
    lyr = folium.GeoJson(
        gdf,
        name='CETESB - Pontos Monitoramento',
        smooth_factor=1.0,
        style_function=lambda x: {
            'fillColor': '#DC143C',
            'color': '#DC143C',
            'weight': 1,
            'fillOpacity': 0.3,
        },
        highlight_function=lambda x: {
            'weight': 3,
            'fillOpacity': 0.6,
        },
        # popup=folium.GeoJsonPopup(
        #     ['popup'],
        #     parse_html=False,
        #     max_width='400',
        #     show=False,
        #     labels=False,
        #     sticky=True,            
        # ),
        marker=folium.Marker(
            icon=folium.Icon(
                color='lightgray',
                icon_color='#FFFF00',
                #icon='leaf',
            ),
        ),
        tooltip=folium.GeoJsonTooltip(
            fields=['cod_ponto'],
            aliases=['Ponto'],
            sticky=True,
            opacity=0.9,
            direction='right',
        ),
        embed=False,
        zoom_on_click=False,
        control=True,
        show=True,
    )
    return lyr


# PopUp
def popup_html_est_aut_empresas(row):
    # Data
    nome = row['Nome']
    posto = row['Posto']
    url_1 = row['url']
    url_2 = row['url_mapa']
    url_3 = row['url_infoposto']
	
    # Infos
    popup = """
    <div>
    <h5>{}</h5>
    <br>Posto: <b>{}</b>
    <br><a href="{}" target="_blank">url - dados</a></b>
    <br><a href="{}" target="_blank">url - mapa</a></b>
    <br><a href="{}" target="_blank">url - infoposto</a></b>
    </div>
    """.format(nome, posto, url_1, url_2, url_3)
    return popup

In [22]:
def add_lyr_ptos_monitoramento_cluster():
    # Input
    gdf = gpd.read_file(os.path.join('..', 'data', 'geo', 'ptos_monitoramento.geojson'))
    gdf = gdf.to_crs(epsg=4326)
    
    # Callbacks
    callback = """
        function (row) {
        var marker = L.marker(new L.LatLng(row[0], row[1]), {color: "red"});

        // Icon
        var icon = L.AwesomeMarkers.icon({
        icon: 'info-sign',
        iconColor: 'white',
        markerColor: 'green',
        prefix: 'glyphicon',
        extraClasses: 'fa-rotate-0'});

        // Popup
        const display_text = {text: row[2]};
        var mytext = $("<div id='my_text' class='display_text' style='width: 100.0%; height: 100.0%;'>${display_text.text}</div>")[0];
        var popup = L.popup({maxWidth: '300'});
        popup.setContent(my_text);
        marker.bindPopup(popup);

        // Hover
        marker.bindPopup(row[2]);
        marker.on('mouseover', function (e) {
            this.openPopup();
        });
        marker.on('mouseout', function (e) {
            this.closePopup();
        });    

        // Results
        return marker
        };
    """

    # Layer
    lyr = plugins.FastMarkerCluster(
        df[['latitude_dd', 'longitude_dd', 'cod_ponto']].values.tolist(),
        callback=callback,
        name='Pontos de Monitoramento',
    )
    return lyr


# PopUp
def popup_html_est_aut_empresas(row):
    # Data
    nome = row['Nome']
    posto = row['Posto']
    url_1 = row['url']
    url_2 = row['url_mapa']
    url_3 = row['url_infoposto']
	
    # Infos
    popup = """
    <div>
    <h5>{}</h5>
    <br>Posto: <b>{}</b>
    <br><a href="{}" target="_blank">url - dados</a></b>
    <br><a href="{}" target="_blank">url - mapa</a></b>
    <br><a href="{}" target="_blank">url - infoposto</a></b>
    </div>
    """.format(nome, posto, url_1, url_2, url_3)
    return popup

<br>

## Basemaps

In [23]:
def add_lyr_google_streets(min_zoom, max_zoom):
    row={
        'link': 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        'name': 'Google Streets',
        'attribution': 'https://www.google.com/maps',
    }
    lyr = folium.TileLayer(
        tiles=row['link'],
        attr=('<a href="{}" target="blank">{}</a>'.format(row['attribution'], row['name'])),
        name=row['name'],
        min_zoom=min_zoom,
        max_zoom=max_zoom,
        subdomains=['mt0','mt1','mt2','mt3'],
        overlay=False,
        control=True,
        show=False,
    )
    return lyr

In [24]:
def add_lyr_google_terrain(min_zoom, max_zoom):
    row={
        'link': 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        'name': 'Google Terrain',
        'attribution': 'https://www.google.com/maps',
    }
    lyr = folium.TileLayer(
        tiles=row['link'],
        attr=('<a href="{}" target="blank">{}</a>'.format(row['attribution'], row['name'])),
        name=row['name'],
        min_zoom=min_zoom,
        max_zoom=max_zoom,
        subdomains=['mt0','mt1','mt2','mt3'],
        overlay=False,
        control=True,
        show=False,
    )
    return lyr

In [25]:
def add_lyr_google_hybrid(min_zoom, max_zoom):
    row={
        'link': 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        'name': 'Google Hybrid',
        'attribution': 'https://www.google.com/maps',
    }
    lyr = folium.TileLayer(
        tiles=row['link'],
        attr=('<a href="{}" target="blank">{}</a>'.format(row['attribution'], row['name'])),
        name=row['name'],
        min_zoom=min_zoom,
        max_zoom=max_zoom,
        subdomains=['mt0','mt1','mt2','mt3'],
        overlay=False,
        control=True,
        show=True,
    )
    return lyr

In [26]:
def add_lyr_google_satelite(min_zoom, max_zoom):
    row={
        'link': 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        'name': 'Google Satelite',
        'attribution': 'https://www.google.com/maps',
    }
    lyr = folium.TileLayer(
        tiles=row['link'],
        attr=('<a href="{}" target="blank">{}</a>'.format(row['attribution'], row['name'])),
        name=row['name'],
        min_zoom=min_zoom,
        max_zoom=max_zoom,
        subdomains=['mt0','mt1','mt2','mt3'],
        overlay=False,
        control=True,
        show=False,
    )
    return lyr

<br>

# Mapa

<br>

## Função

In [27]:
def get_map(input_geojson):
    # Input
    gdf = gpd.read_file(input_geojson)
    gdf = gdf.to_crs(epsg=4326)
    sw = gdf.bounds[['miny', 'minx']].min().values.tolist()
    ne = gdf.bounds[['maxy', 'maxx']].max().values.tolist()
    bounds = [sw, ne]
    
    # Zoom
    min_zoom = 6
    max_zoom = 21
    padding = 1
    
    # Create Map
    m = folium.Map(
        min_zoom=min_zoom,
        max_zoom=max_zoom,
        max_bounds=True,
        min_lat=bounds[0][0]*((100+padding)/100),
        min_lon=bounds[0][1]*((100+padding)/100),
        max_lat=bounds[1][0]*((100-padding)/100),
        max_lon=bounds[1][1]*((100-padding)/100),
        tiles=None,
        # zoom_delta=0.1,
        # zoom_start=10,
    )
    
    # Add Layers
    m.add_child(add_lyr_google_terrain(min_zoom, max_zoom))
    m.add_child(add_lyr_google_hybrid(min_zoom, max_zoom))
    m.add_child(add_lyr_google_streets(min_zoom, max_zoom))
    m.add_child(add_lyr_google_satelite(min_zoom, max_zoom))

    # Monitoramento
    #m.add_child(add_lyr_ptos_monitoramento())
    m.add_child(add_lyr_ptos_monitoramento_cluster())
    #m.add_child(add_lyr_est_aut_empresas())

    # Bacse PCJ
    #m.add_child(add_lyr_municipios_pcj(base_pcj_path))
    #m.add_child(add_lyr_bacias_pcj(base_pcj_path))    
    
    #feature_group = folium.FeatureGroup('Hidrografia')
    #feature_group.add_child(add_lyr_hidrografia_principal(base_pcj_path))
    #feature_group.add_child(add_lyr_represas_secundarias(base_pcj_path)) 
    #feature_group.add_child(add_lyr_represas_principais(base_pcj_path))
    #feature_group.add_to(m)

    # Plugins
    m.fit_bounds(bounds)
    plugins.Fullscreen(
        position='topleft',
        title='Clique para Maximizar',
        title_cancel='Mininizar',
    ).add_to(m)
    folium.LayerControl(
        position='topright',
        collapsed=False,
    ).add_to(m)
    return m

In [28]:
# Create Maps
m = get_map(os.path.join('..', 'data', 'geo', 'ptos_monitoramento.geojson'))

# Output Path
output_path = os.path.join('..', 'maps')
os.makedirs(output_path, exist_ok=True)

# Save
m.save(os.path.join(output_path, 'map.html'))
m.save(os.path.join(os.path.expanduser('~/Documents/Sourcecode'), 'gitpages', 'open-geodata.github.io', 'assets', 'sp_cetesp_infoaguas', 'map_infoaguas.html'))

# Results
m

<br>

## Cluster

In [29]:
# Input
input_geojson = os.path.join('..', 'data', 'geo', 'ptos_monitoramento.geojson')
gdf = gpd.read_file(input_geojson)
gdf = gdf.to_crs(epsg=4326)
sw = gdf.bounds[['miny', 'minx']].min().values.tolist()
ne = gdf.bounds[['maxy', 'maxx']].max().values.tolist()
bounds = [sw, ne]

In [30]:
# Cria o mapa
m = folium.Map(
    location=[-23.9619271,-46.3427499],
    zoom_start=12,
)

# Callbacks
callback = """
    function (row) {
    
    // Icon
    var greenIcon = L.icon({
        iconUrl: 'http://leafletjs.com/examples/custom-icons/leaf-green.png',
        sshadowUrl: 'http://leafletjs.com/examples/custom-icons/leaf-shadow.png',
        iconSize:     [38, 95], // size of the icon
        shadowSize:   [50, 64], // size of the shadow
        iconAnchor:   [22, 94], // point of the icon which will correspond to marker's location
        shadowAnchor: [4, 62],  // the same for the shadow
        popupAnchor:  [-3, -76] // point from which the popup should open relative to the iconAnchor
    });
        
    // Icon
    var icon = L.AwesomeMarkers.icon({
            icon: 'info-sign',
            iconColor: 'white',
            markerColor: 'blue',
            prefix: 'glyphicon',
            extraClasses: 'fa-rotate-0'
    });
    
    // Markers
    var marker = L.marker(new L.LatLng(row[0], row[1]), {icon: icon});
    
    // Popup
    const display_text = {text: row[2]};
    var my_text = $(`<div id='my_text' class='display_text' style='width: 100.0%; height: 100.0%;'>${display_text.text}<br></div>`)[0];
    var popup = L.popup({maxWidth: '300'});
    popup.setContent(my_text);
    marker.bindPopup(popup);
    
    // Hover
    marker.bindTooltip(row[2]);
    marker.on('mouseover', function (e) {
        this.openTooltip();
    });
    marker.on('mouseout', function (e) {
        this.closeTooltip();
        //console.log('Michellll');
    });
    
    // Results
    return marker
    };
"""

# dddd
plugins.FastMarkerCluster(
    df[['latitude_dd', 'longitude_dd', 'cod_ponto']].values.tolist(),
    callback=callback,
    name='Pontos de Monitoramento',
    overlay=True,
    control=True,
    show=True,    
).add_to(m)

# Plugins
m.fit_bounds(bounds)
plugins.Fullscreen(
    position='topleft',
    title='Clique para Maximizar',
    title_cancel='Mininizar',
).add_to(m)
folium.LayerControl(
    position='topright',
    collapsed=False,
).add_to(m)

# Results
m.save(os.path.join(output_path, 'map_cluster.html'))
m.save(os.path.join(os.path.expanduser('~/Documents/Sourcecode'), 'gitpages', 'open-geodata.github.io', 'assets', 'sp_cetesp_infoaguas', 'map_infoaguas_cluster.html'))

m

<br>

## Single

In [31]:
# Cria o mapa
m = folium.Map(
    location=[-23.9619271,-46.3427499],
    zoom_start=12,
    #width=767,
    #height=460,
)

# Adiciona as diferentes empresas com cores por bairros
for index, row in df.iterrows():
    #if row['sistema_hidrico'] in colors.keys():
    folium.Marker(
        location=[row['latitude_dd'], row['longitude_dd']],
        #popup=row['sistema_hidrico'],
        #tooltip=row['localizacao'],
        #icon=folium.Icon(color=colors[row[col_categories]], icon='leaf')
    ).add_to(m)

# Results
m.save(os.path.join(output_path, 'map_single.html'))
m